In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# 11. cfp
# 季度频率。营运现金流除以季度末市值。

# 12. cfp_ia
# 季度频率。经行业调整后的营运现金流。调整方法与bm_ia类似。

In [3]:
import pandas as pd
import numpy as np
from utils.diff import diff  # 确保 diff 函数正确导入

# 读取现金流量表
df1 = pd.read_csv('季_现金流量表(直接法).csv', usecols=['证券代码', '会计期间', '经营活动产生的现金流量净额'], low_memory=False)

# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

# 读取相对价值指标
df2 = pd.read_csv('季_相对价值指标.csv', usecols=['证券代码', '会计期间', '市值A'])

# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

# 合并数据
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')

# 计算差异
df3 = diff(df3, col=['经营活动产生的现金流量净额'])

# 计算现金流占市值比例
df3['cfp'] = df3['经营活动产生的现金流量净额'] / df3['市值A'].replace(0, np.nan)

# 读取公司文件
df4 = pd.read_csv('公司文件.csv', usecols=['证券代码', '行业代码C'])

# 确保证券代码为整数类型
df4['证券代码'] = pd.to_numeric(df4['证券代码'], errors='coerce').fillna(0).astype(int)

# 合并数据
df5 = pd.merge(df3, df4, on=['证券代码'], how='left')

# 计算行业平均
df6 = df5[['行业代码C', '会计期间', 'cfp']].groupby(['行业代码C', '会计期间']).mean().reset_index()
df6.rename(columns={'cfp': 'cfp_i'}, inplace=True)

# 合并行业平均
df7 = pd.merge(df5, df6, on=['行业代码C', '会计期间'], how='left')

# 计算差异
df7['cfp_ia'] = df7['cfp'] - df7['cfp_i']

# 选择最终列并导出
df8 = df7[['证券代码', '会计期间', 'cfp', 'cfp_ia']]
df8.to_csv('季_11_12.csv', index=False, encoding='utf-8-sig')

# 输出结果 DataFrame
print(df8)

          证券代码        会计期间  cfp  cfp_ia
0            1  2000-03-31  NaN     NaN
1            1  2000-06-30  NaN     NaN
2            1  2000-09-30  NaN     NaN
3            1  2000-12-31  NaN     NaN
4            1  2001-03-31  NaN     NaN
...        ...         ...  ...     ...
571126  920118  2023-09-30  NaN     NaN
571127  920118  2023-12-31  NaN     NaN
571128  920118  2024-03-31  NaN     NaN
571129  920118  2024-06-30  NaN     NaN
571130  920118  2024-09-30  NaN     NaN

[571131 rows x 4 columns]
